In [1]:
from dask.distributed import Client
client = Client()

In [3]:
import duckdb
cursor = duckdb.connect()

In [5]:
import dask.dataframe as ddf

In [10]:
neo_raw = ddf.read_parquet('/tmp/data/neo_parquet/*/*.parquet')

In [61]:
neo_hazardous = neo_raw.loc[neo_raw['is_potentially_hazardous_asteroid']].compute()

In [64]:
neo_hazardous_final = neo_hazardous[['id', 'name','close_approach_date','miss_distance_miles']]

In [65]:
neo_hazardous_final

,id,name,close_approach_date,miss_distance_miles
0,2216523,216523 (2001 HY7),2021-04-02,29425246.37342055
0,2004581,4581 Asclepius (1989 FC),2021-04-03,26116943.6698277706
2,2326290,326290 Akhenaten (1998 HE3),2021-04-03,24495566.5599161034
3,2514596,514596 (2003 FG),2021-04-03,6043170.9397648978
4,2523804,523804 (2000 YF29),2021-04-03,10104656.9294565696
2,3610527,(2012 TP139),2021-04-05,12328976.2677074094
5,54054354,(2020 RH7),2021-04-06,24400158.7377786936


In [70]:
neo_hazardous_final.to_parquet(path='/tmp/data/neo_hazardous_result.parquet')

In [72]:
cursor.execute("CREATE OR REPLACE VIEW neo_data AS SELECT * FROM parquet_scan('/tmp/data/neo_parquet/*/*.parquet')").fetchall();

In [74]:
print(cursor.execute("SELECT id,name,miss_distance_miles FROM neo_data where is_potentially_hazardous_asteroid = True").fetchall())

[('2004581', '4581 Asclepius (1989 FC)', '26116943.6698277706'), ('2326290', '326290 Akhenaten (1998 HE3)', '24495566.5599161034'), ('2514596', '514596 (2003 FG)', '6043170.9397648978'), ('2523804', '523804 (2000 YF29)', '10104656.9294565696'), ('54054354', '(2020 RH7)', '24400158.7377786936'), ('3610527', '(2012 TP139)', '12328976.2677074094'), ('2216523', '216523 (2001 HY7)', '29425246.37342055')]
